In [6]:
import duckdb

offline_analysis_id = 15

con = duckdb.connect("duck_db_analysis_database.db", read_only= True)
con2 = duckdb.connect(f"offline_analysis_id_{offline_analysis_id}.duckdb")

In [7]:
def add_pgf_meta_raw_tables_batch(experiment_series, results,  con , con2):
    table_names = ["sweep_table_name", "meta_data_table_name", "pgf_data_table_name"]
    for name in table_names:
        for table in experiment_series[name]:
                df_data = con.execute(f"Select * from {table}").fetchdf()
                con2.execute(f"CREATE TABLE {table} AS SELECT * FROM df_data")
    
    for result in results["specific_result_table_name"]:
        df_data = con.execute(f"Select * from {result}").fetchdf()
        con2.execute(f"CREATE TABLE {result} AS SELECT * FROM df_data")


def create_table_offline_analysis(con2, offline_analysis_id, max_length):

    con2.execute(f"CREATE SEQUENCE unique_offline_analysis_sequence START {offline_analysis_id + 1};")
    con2.execute(f"CREATE SEQUENCE analysis_function_sequence START {max_length + 1};")

    sql_create_offline_analysis_table = """ CREATE TABLE offline_analysis(
                                            analysis_id integer PRIMARY KEY DEFAULT(nextval ('unique_offline_analysis_sequence')),
                                            date_time TIMESTAMP,
                                            user_name TEXT,
                                            selected_meta_data text); """
    
    sql_create_analysis_function_table = """ CREATE TABLE analysis_functions(
                                            analysis_function_id integer PRIMARY KEY DEFAULT(NEXTVAL('analysis_function_sequence')),
                                            function_name text,
                                            lower_bound float,
                                            upper_bound float,
                                            analysis_series_name text,
                                            analysis_id integer
                                            );"""

    con2.execute(sql_create_offline_analysis_table)
    con2.execute(sql_create_analysis_function_table)


#create_table_offline_analysis(con2, offline_analysis_id)
# get the analysis_series_name from the duckdb database using a certain analysis_id code
# this should be a export offline analysis id function that can be shared and used with others using the viewer portion!
# tables from experiment section
experiment_analysis_mapping = con.execute(f"SELECT * FROM experiment_analysis_mapping WHERE analysis_id = {offline_analysis_id}").fetchdf() # experiment_analysis_mapping_table
experiments = con.execute(f"SELECT * FROM experiments WHERE experiment_name IN {tuple(experiment_analysis_mapping['experiment_name'].values)}").fetch_df() # experiment_table
experiment_series = con.execute(f"SELECT * FROM experiment_series WHERE experiment_name IN {tuple(experiment_analysis_mapping['experiment_name'].values)}").fetch_df()

# Analysis Tables
offline_analysis = con.execute(f"SELECT * FROM offline_analysis WHERE analysis_id = {offline_analysis_id}").fetchdf()
analysis_series =  con.execute(f"SELECT * FROM analysis_series WHERE analysis_id = {offline_analysis_id}").fetchdf()
analysis_functions =  con.execute(f"SELECT * FROM analysis_functions WHERE analysis_id = {offline_analysis_id}").fetchdf()
max_index = max(analysis_functions["analysis_function_id"].tolist())
create_table_offline_analysis(con2, offline_analysis_id, max_index)
# result table names
results =  con.execute(f"SELECT * FROM results WHERE analysis_id = {offline_analysis_id}").fetchdf()

#here we retrieve the selected metadata tables from the analysis
global_meta_data =  con.execute(f"SELECT * FROM global_meta_data WHERE experiment_name IN {tuple(experiment_analysis_mapping['experiment_name'].values)}").fetchdf()
selected_meta_data =  con.execute(f"SELECT * FROM selected_meta_data WHERE offline_analysis_id = {offline_analysis_id}").fetchdf()
sweep_meta_data =  con.execute(f"SELECT * FROM sweep_meta_data WHERE experiment_name IN {tuple(experiment_analysis_mapping['experiment_name'].values)}").fetchdf()


In [8]:
analysis_series

,analysis_series_name,time,recording_mode,analysis_id
0,Block Pulse,NaN,Voltage Clamp,15
1,IV,NaN,Voltage Clamp,15
2,IV_40_new,NaN,Voltage Clamp,15
3,Inact,NaN,Voltage Clamp,15
4,CClamp,NaN,Current Clamp,15
5,InputRes,NaN,Current Clamp,15


In [2]:
transfer_list = ["experiment_analysis_mapping",
                 "experiments",
                 "experiment_series", 
                 "analysis_series", 
                 "results", 
                 "global_meta_data",
                 "selected_meta_data",
                 "sweep_meta_data"
                ]    
   
transfer_two = ["offline_analysis",
                "analysis_functions"]

for data in transfer_list:
     con2.execute(f"CREATE TABLE {data} AS SELECT * FROM {data}")

con2.executemany("INSERT INTO offline_analysis VALUES (?, ?, ?, ?)", offline_analysis.values)
con2.executemany("INSERT INTO analysis_functions VALUES (?, ?, ?, ?,?,?)", analysis_functions.values)

add_pgf_meta_raw_tables_batch(experiment_series, results, con, con2)

NameError: name 'con2' is not defined

In [10]:
trial = duckdb.connect(r"C:\Users\user\Desktop\duck_db_analysis_database.db")

In [14]:
con.close()